# 1. Import Dependencies


In [ ]:
!pip install mediapipe opencv-python matplotlib tensorflow

In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp


# 2. Keypoints using MP Holistic

In [ ]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # drawing utilities

In [ ]:
def mp_detection(image : np.ndarray, model):
    """ 
        detect the landmark in the image 
        image : opencv frame
        model : holistic model from mediapipe
    """
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # convert color of the image BGR -> RGB
    image.flags.writeable = False # not writable
    results = model.process(image) # detect the holistic 
    image.flags.writeable = True
    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR) # RGB -> BGR
    return image,results

In [ ]:
def draw_styled_landmarks(image, results):
    """
        image : opencv frame/ image
        results : the results of the holistic model from mediapipe use the mp_detection to get the results 
    """
    try : 
        mp_drawing.draw_landmarks(image,results.face_landmarks, mp_holistic.FACE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                )
    except Exception as e:
        pass
    try : 
        mp_drawing.draw_landmarks(image,results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                )
    except Exception as e:
        pass
    try : 
        mp_drawing.draw_landmarks(image,results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                )
    except Exception as e:
        pass
    try : 
        mp_drawing.draw_landmarks(image,results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                )
    except Exception as e:
        pass

In [ ]:
# test access to web cam 

cap = cv2.VideoCapture(0) # device 0

# initialse detection confidence and then the confidence of tracking those detections
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic_model:

    while cap.isOpened():
        ret, frame = cap.read() # read one fream at this point of time

        # detection of the keypoint
        image, resuts = mp_detection(frame,holistic_model)
        

        cv2.imshow("openCV feed cam", frame) # window

        if cv2.waitKey(10) & 0xFF == ord('q'): # wait a key is pressed and check current key is 'q'
            break

    #release ressources
    cap.release()
    cv2.destroyAllWindows()


In [ ]:
draw_styled_landmarks(frame,resuts)
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

### 2.1 Test reading Keypoint from downloaded video

In [ ]:
# test access to video file

cap = cv2.VideoCapture(r"C:\Users\Kokou\Documents\Project\Sign-Language-Detection\data\subset\cont\videos\CLSFB - 01 ok\CLSFBI0103A_S002_B.mp4") # device 0

# initialse detection confidence and then the confidence of tracking those detections
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic_model:

    while cap.isOpened():
        ret, frame = cap.read() # read one fream at this point of time

        # detection of the keypoint
        image, results = mp_detection(frame,holistic_model)
        draw_styled_landmarks(image,results)
        

        cv2.imshow("openCV feed cam", image) # window

        if cv2.waitKey(10) & 0xFF == ord('q'): # wait a key is pressed and check current key is 'q'
            break

    #release ressources
    cap.release()
    cv2.destroyAllWindows()



In [ ]:
draw_styled_landmarks(frame,results)
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

## 3. Extract Keyoint Values

In [ ]:
def extract_keypoints(results):
    """
    Extract all the keypoint from the results

    """
    # if no landmark for the hand => zeros array => no hand in teh frame
    # same idea for others part
    pose = np.array([[res.x,res.y,res.z,res.visibility]  for res in results.pose_landmarks.landmark ]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face  = np.array([[res.x,res.y,res.z]  for res in results.face_landmarks.landmark ]).flatten() if results.face_landmarks else np.zeros(468*3)
    left_hand = np.array([[res.x,res.y,res.z]  for res in results.left_hand_landmarks.landmark ]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    right_hand = np.array([[res.x,res.y,res.z]  for res in results.right_hand_landmarks.landmark ]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, left_hand, right_hand])


In [ ]:

extract_keypoints(results).shape == (468*3+33*4+21*3*2,)

## 4. Setup Folders for collection

In [ ]:
# Using the keypoint to decode tghe sign language 
# set up folder to store keypoint associeted to a frame

DATA_PATH = os.path.join("MP_Data") # path for the data -> numpy arrays

actions = np.array(['aussi', 'oui', 'quoi']) # actions that we try to detect
no_sequences = 100 # 30 videos of data for each actions
sequence_length = 20 # 30 frames in length

# we collect 30 videos per action the each videos contains 30 frames of data 
# and each frame contains 1662 landmark values (3*30 sequences, 30 frames, 1662 landmark)


In [ ]:
# create a folder for each action containing a folder for each fequence
for action in actions:
    for seq in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH,action,str(seq)))
        except OSError as e:
            os.rmdir(os.path.join(DATA_PATH,action,str(seq)))
            try:
                os.makedirs(os.path.join(DATA_PATH,action,str(seq)))
            except:
                pass

## 5.  Collect Keypoint Values for testing and training

In [ ]:
def visio_function():
        # test access to video file

    cap = cv2.VideoCapture(0) # device 0

    # initialse detection confidence and then the confidence of tracking those detections
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic_model:

        for action in actions:
            for sequence in range(no_sequences):
                for frame_num in range(sequence_length):
                        
                    ret, frame = cap.read() # read one fream at this point of time

                    # detection of the keypoint
                    image, results = mp_detection(frame,holistic_model)
                    draw_styled_landmarks(image,results)
                    
                    if frame_num == 0:
                        cv2.putText(image, "STARTING COLLECTION", (120,200), cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),4,cv2.LINE_AA)
                        cv2.putText(image, f"Collection frames for {action} video Number {sequence}", (15,12), cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
                        cv2.imshow("openCV feed cam", image) # window
                        
                        cv2.waitKey(2000)
                    else:
                        cv2.putText(image, f"Collection frames for {action} video Number {sequence}", (15,12), cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)

                        cv2.imshow("openCV feed cam", image) # window
                    
                    #save keypoint in file
                    keypoint = extract_keypoints(results)
                    np_file_path = os.path.join(DATA_PATH,action,str(sequence), str(frame_num))
                    np.save(np_file_path,keypoint)

                    if cv2.waitKey(10) & 0xFF == ord('q'): # wait a key is pressed and check current key is 'q'
                        break

            #release ressources
            cap.release()
            cv2.destroyAllWindows()



In [ ]:
#Read video files downloaded (isol files) and get the landmark associeted 
BASE_PATH = "./../data/subset/isol/videos/"

# initialse detection confidence and then the confidence of tracking those detections
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic_model:

    for action in actions:
        video_files = os.listdir(os.path.join(BASE_PATH,"_"+str(action).upper()+"_"))
        sequence = 0
        #loop through videos aka sequences
        while sequence != no_sequences:
            file = os.path.join(BASE_PATH,"_"+str(action).upper()+"_",video_files[sequence])
            cap = cv2.VideoCapture(file) # device 0

            for frame_num in range(sequence_length):
                    
                ret, frame = cap.read() # read one fream at this point of time

                # detection of the keypoint
                if not (frame is None) : # save the last keypoint if the nb of frame is lower than sequence length
                    image, results = mp_detection(frame,holistic_model)
                    draw_styled_landmarks(image,results)
                    # cv2.putText(image, f"Collection frames for {action} video Number {sequence}", (15,12), cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
                    # cv2.imshow("openCV feed", image) # window
                
                #save keypoint in file
                keypoints = extract_keypoints(results)
                if keypoints.shape != (1662,): #debug
                    print(keypoints.shape)
                np_file_path = os.path.join(DATA_PATH,action,str(sequence), str(frame_num))
                np.save(np_file_path,keypoints)

                if cv2.waitKey(10) & 0xFF == ord('q'): # wait a key is pressed and check current key is 'q'
                    break
            sequence +=1 # next file

    #release ressources
    cap.release()
    cv2.destroyAllWindows()



In [ ]:
cap.release()
cv2.destroyAllWindows()

## 6. Preprocess Data and creeate Labels and features

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical #convert data into one hot encoded data

In [ ]:
label_map = {label:num for num,label in enumerate(actions)}
sequences = [] # represent the feature data X
labels = [] # represente the Y data

for action in actions:
    for sequence in range(no_sequences):
        window = []
        subfolder = os.path.join(DATA_PATH,action,str(sequence))
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(subfolder, f"{frame_num}.npy"))
            window.append(res)
            
            if res.shape != (1662,): #debug
                print(res.shape)
        sequences.append(window) # containt 90 differente video 3 words * 30 videos
        labels.append(label_map[action])



In [ ]:
np.array(sequences).shape

In [ ]:
X = np.array(sequences)
y = to_categorical(labels).astype(int) # convert label to binary version

In [ ]:
X_train, X_test, y_train, y_test, = train_test_split(X,y,test_size=0.05)

In [ ]:
y_train.shape

## 7. Build and Train LSTM Neural Network

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
# tensorBoard logs
log_dir = os.path.join("Logs")
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=True,activation='relu', input_shape=(sequence_length,1662)))
model.add(LSTM(128, return_sequences=True,activation='relu'))
model.add(LSTM(64, return_sequences=False,activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))


In [ ]:
# categorical_crossentropy => for multi-class classification model
# metrics => allow to track the accuracy 
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(X_train,y_train,epochs=2000,callbacks=[tb_callback])
# 2000 epoch may be a bit high for training with a low amount of data
# stop the training if accuracy is acceptable and loss has stoppped consistently decreasing 

## 8. Make Prediction

In [ ]:
res = model.predict(X_test)
actions[np.argmax(res[0])]

In [ ]:
actions[np.argmax(y_test[0])]

In [ ]:
model.save('action.h1')
model.load_weights('action.h1')

## 9. Evaluation and accuracy

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix,accuracy_score

yhat = model.predict(X_test)
ytrue = np.argmax(y_test,axis=1).tolist()
yhat = np.argmax(yhat,axis=1).tolist()
multilabel_confusion_matrix(ytrue,yhat)

In [ ]:
accuracy_score(ytrue,yhat)

## 10. Test in Real Time

In [ ]:

colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    """
    probability for each of the three words
    """

    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame



In [ ]:
sequence = []
sentence = []
threshold = 0.4 # threshold to accept a sign

cap = cv2.VideoCapture(0) # device 0

# initialse detection confidence and then the confidence of tracking those detections
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic_model:

    while cap.isOpened():
            
        ret, frame = cap.read() # read one fream at this point of time

        # detection of the keypoint
        image, results = mp_detection(frame,holistic_model)
        draw_styled_landmarks(image,results)
        
        # prediction using LSTM model
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]

        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence,axis=0))[0]
            print(actions[np.argmax(res)])
            
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]: # add to sentences only differente word
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)


        #save keypoint in file
        cv2.imshow("openCV feed cam", image) # window
        
        if cv2.waitKey(10) & 0xFF == ord('q'): # wait a key is pressed and check current key is 'q'
            break

    #release ressources
    cap.release()
    cv2.destroyAllWindows()

